In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import time
import torchvision
import sys
import os
from argparse import ArgumentParser
from torchvision.datasets import CIFAR10 as data_cifar
from torchvision.transforms import Normalize, ToTensor, Compose, RandomCrop, RandomHorizontalFlip
from easydict import EasyDict
from torch.optim.lr_scheduler import MultiStepLR
import resnet

In [12]:
args = EasyDict({"lr": 0.1, "resume":False})

transform_train = Compose([
    RandomCrop(32, padding=4),
    RandomHorizontalFlip(),
    ToTensor(),
    Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

transform_test = Compose([
    ToTensor(),
    Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

trainset = data_cifar(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=4)

testset = data_cifar(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [15]:
print("The length of training set is", len(trainset))
print("The length of testing set is:", len(testset))

The length of training set is 50000
The length of testing set is: 10000


In [16]:
net = resnet.ResNetUse()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args.lr ,
                      momentum=0.9, weight_decay = 1e-4)

# Training
scheduler = MultiStepLR(optimizer, milestones=[30,40], gamma=0.1)

def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
      inputs, targets = inputs.to(device), targets.to(device)
      optimizer.zero_grad()
      outputs = net(inputs)
      loss = criterion(outputs, targets)
      loss.backward()
      optimizer.step()
      train_loss += loss.item()
      _, predicted = outputs.max(1)
      total += targets.size(0)
      correct += predicted.eq(targets).sum().item()
    print("The training accuracy is:", correct/total)
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    print("The testing accuracy is:", correct/total)

for epoch in range(0, 50):
    print('current lr {:.5e}'.format(optimizer.param_groups[0]['lr']))
    train(epoch)
    scheduler.step()
    test(epoch)

current lr 1.00000e-01

Epoch: 0


KeyboardInterrupt: 